In [1]:
# coding=utf-8

# 导入模块
from __future__ import print_function
from __future__ import absolute_import

import os
import json
import warnings
import keras
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from keras import backend
from keras import layers
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, Activation, Dropout, Flatten, Dense
from keras.layers import BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, PReLU
from keras.layers import GlobalAveragePooling2D, GlobalMaxPool2D
from keras.optimizers import SGD, Adam
from keras.callbacks import LearningRateScheduler, ModelCheckpoint, TensorBoard, EarlyStopping, ReduceLROnPlateau

import numpy as np
import json

warnings.filterwarnings("ignore")

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


# 参数
img_width = 300
img_height = 200  # 图片的长宽
train_data_dir = "../TCM/train/"  # 训练集的文件位置
test_data_dir = "../TCM/val/"  # 测试集的文件位置

epochs = 60 # epoch数目
batch_size = 8  # 批次大小
classes = 50 # 样本分类数目
model_name = "idadp_test_3.h5" # model的名称

#卷积层Conv Block
def conv_block(x, growth_rate, name):
    bn_axis = 3
    x1 = layers.BatchNormalization(axis=bn_axis,
                                   epsilon=1.001e-5,
                                   name=name + "_0_bn")(x)  # 批量标准化层
    x1 = layers.Activation("relu", name=name + "_0_relu")(x1)  # 激活
    x1 = layers.Conv2D(4 * growth_rate,
                       1,
                       use_bias=False,
                       name=name + "_1_conv")(x1)  # 2D卷积核

    x1 = layers.BatchNormalization(axis=bn_axis,
                                   epsilon=1.001e-5,
                                   name=name + "_1_bn")(x1)
    x1 = layers.Activation("relu", name=name + "_1_relu")(x1)
    x1 = layers.Conv2D(growth_rate,
                       3,
                       padding='same',
                       use_bias=False,
                       name=name + "_2_conv")(x1)
    x = layers.Concatenate(axis=bn_axis, name=name + "_concate")([x, x1])
    return x

#Dense Block
#卷积层的叠加, 每一个卷积层都与其之上的同一个DenseBlock中的卷积层有相互链接
def dense_block(x, blocks, name):
    for i in range(blocks):
        x = conv_block(x, 32, name=name + "_block" + str(i + 1))
    return x

#Transition Block
#连接每个Dense Block
def transition_block(x, reduction, name):
    bn_axis = 3
    x = layers.BatchNormalization(axis=bn_axis,
                                  epsilon=1.001e-5,
                                  name=name + "_0_bn")(x)
    x = layers.Activation('relu', name=name + "_relu")(x)
    x = layers.Conv2D(int(backend.int_shape(x)[bn_axis] * reduction),
                      1,
                      use_bias=False,
                      name=name + "_conv")(x)
    x = layers.AveragePooling2D(2, strides=2, name=name + "_pool")(x)
    return x

# DenseNet的实现
def dense_net(blocks, input_shape=(600, 400, 3), classes=6):
    img_input = Input(shape=input_shape)

    bn_axis = 3

    x = layers.ZeroPadding2D(padding=((3, 3), (3, 3)))(img_input)
    x = layers.Conv2D(64, 7, strides=2, use_bias=False, name="conv1/conv")(x)
    x = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name="conv1/bn")(x)
    x = layers.Activation("relu", name="conv1/relu")(x)
    x = layers.ZeroPadding2D(padding=((1, 1), (1, 1)))(x)
    x = layers.MaxPooling2D(3, strides=2, name="pool1")(x)
    
    x = dense_block(x, blocks[0], name="conv2")
    x = transition_block(x, 0.5, name="pool2")
    x = dense_block(x, blocks[1], name="conv3")
    x = transition_block(x, 0.5, name="pool3")
    x = dense_block(x, blocks[2], name="conv4")
    x = transition_block(x, 0.5, name="pool4")
    x = dense_block(x, blocks[3], name="conv5")
    
    x = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name="bn")(x)
    x = layers.GlobalAveragePooling2D(name="avg_pool")(x)
    
    x = Dense(512)(x)
    x = BatchNormalization()(x)
    x = PReLU()(x)
    x = Dropout(0.5)(x)
    x = Dense(classes, activation="softmax", name="fc6")(x)
    
    inputs = img_input
    model = Model(inputs, x, name="densenet")
    
    return model

model = dense_net(blocks=[6, 12, 48, 32], input_shape=(300, 200, 3), classes=50)
model.summary() # 输出网络详细信息

Using TensorFlow backend.
/home/hyq-user/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/hyq-user/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/hyq-user/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/hyq-user/.local/lib/python3.5/site-packages/tensorflo

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 300, 200, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 306, 206, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 150, 100, 64) 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 150, 100, 64) 256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

conv5_block30_1_conv (Conv2D)   (None, 9, 6, 128)    233472      conv5_block30_0_relu[0][0]       
__________________________________________________________________________________________________
conv5_block30_1_bn (BatchNormal (None, 9, 6, 128)    512         conv5_block30_1_conv[0][0]       
__________________________________________________________________________________________________
conv5_block30_1_relu (Activatio (None, 9, 6, 128)    0           conv5_block30_1_bn[0][0]         
__________________________________________________________________________________________________
conv5_block30_2_conv (Conv2D)   (None, 9, 6, 32)     36864       conv5_block30_1_relu[0][0]       
__________________________________________________________________________________________________
conv5_block30_concate (Concaten (None, 9, 6, 1856)   0           conv5_block29_concate[0][0]      
                                                                 conv5_block30_2_conv[0][0]       
__________

In [2]:
# 测试模型
model.load_weights("./idadp_test_2.h5") # 读取模型

In [3]:
import os
train_dir = "../TCM/train/"
test_dir = "../TCM/test/"
val_dir = "../TCM/val/"
print(os.listdir(test_dir))

['19', '31', '28', '4', '25', '15', '47', '22', '10', '7', '43', '9', '5', '18', '16', '21', '26', '37', '49', '48', '24', '14', '11', '17', '3', '46', '8', '40', '6', '45', '33', '30', '39', '50', '2', '13', '12', '1', '27', '42', '41', '35', '29', '23', '20', '38', '32', '36', '44', '34']


In [4]:
def get_result(file_dir):
    class_index = [1,10,11,12,13,14,15,16,17,18,19,
                   2,20,21,22,23,24,25,26,27,28,29,
                   3,30,31,32,33,34,35,36,37,38,39,
                   4,40,41,42,43,44,45,46,47,48,49,
                   5,50,
                   6,
                   7,
                   8,
                   9]
    images = os.listdir(file_dir)
    result = []
    for image_name in images:
        image_path = file_dir + image_name  # 单个测试图片位置
        image_file = image.load_img(image_path,
                                target_size=(300, 200))
        x = image.img_to_array(image_file) / 255.0
        x = np.expand_dims(x, axis=0)
        prediction = model.predict(x)
        temp_json = dict()
        temp_json["image_id"] = image_name
        temp_json["category"] = class_index[int(np.argmax(prediction, axis=1))]
        result.append(temp_json)
    return result

def json_output(result, file_name):
    json2 = json.dumps(result)
    f = open(file_name,'w',encoding='utf-8')
    f.write(json2)
    f.close()

In [5]:
def get_test_acc(file_dir, category):
    class_index = [1,10,11,12,13,14,15,16,17,18,19,
                   2,20,21,22,23,24,25,26,27,28,29,
                   3,30,31,32,33,34,35,36,37,38,39,
                   4,40,41,42,43,44,45,46,47,48,49,
                   5,50,
                   6,
                   7,
                   8,
                   9]
    images = os.listdir(file_dir)
    result = []
    count = 0
    correct = 0
    for image_name in images:
        path = file_dir + image_name
        file = image.load_img(path, target_size=(300, 200))
        x = image.img_to_array(file) / 255.0
        x = np.expand_dims(x, axis=0)
        prediction = class_index[int(np.argmax(model.predict(x), axis=1))]
        count = count + 1
        if prediction is category:
            correct = correct + 1
#     print(correct, "/", count)
    acc = correct / count
    return acc

In [12]:
def test1(file_dir, classes):
    sum = 0
    f = open("test_acc.txt", "w+")
    for i in range(1, classes + 1):
        temp_acc = get_test_acc(file_dir + str(i) + "/", i)
        print("category", i, ":", temp_acc)
        print("category", i, ":", temp_acc, file=f)
        sum = sum + temp_acc
    print("test_acc:", sum / classes)

def test2(file_dir, classes):
    for i in range(1, classes + 1):
        get_result(file_dir + str(i) + "/")

In [10]:
i = 21
get_test_acc(test_dir + str(i) + "/", i)

KeyboardInterrupt: 

In [15]:
test1(test_dir, 50)

category 1 : 0.8666666666666667
category 2 : 0.9111111111111111
category 3 : 1.0
category 4 : 1.0
category 5 : 1.0
category 6 : 1.0
category 7 : 0.8222222222222222
category 8 : 0.9777777777777777
category 9 : 1.0
category 10 : 1.0
category 11 : 1.0
category 12 : 1.0
category 13 : 1.0
category 14 : 0.9333333333333333
category 15 : 1.0
category 16 : 0.8444444444444444
category 17 : 1.0
category 18 : 0.6444444444444445
category 19 : 0.9777777777777777
category 20 : 0.9555555555555556
category 21 : 0.4888888888888889
category 22 : 0.9777777777777777
category 23 : 1.0
category 24 : 1.0
category 25 : 1.0
category 26 : 0.9555555555555556
category 27 : 0.9333333333333333
category 28 : 0.9333333333333333
category 29 : 0.9777777777777777
category 30 : 1.0
category 31 : 1.0
category 32 : 1.0
category 33 : 1.0
category 34 : 0.9777777777777777
category 35 : 0.9777777777777777
category 36 : 0.9777777777777777
category 37 : 0.675
category 38 : 1.0
category 39 : 0.8
category 40 : 0.9777777777777777
ca

[{'category': 21, 'image_id': 'IMG_20190711_092246.jpg'},
 {'category': 15, 'image_id': 'IMG_20190711_093141.jpg'},
 {'category': 15, 'image_id': 'IMG_20190711_093041.jpg'},
 {'category': 21, 'image_id': 'IMG_20190711_093444.jpg'},
 {'category': 15, 'image_id': 'IMG_20190711_093135.jpg'},
 {'category': 21, 'image_id': 'IMG_20190711_094208.jpg'},
 {'category': 21, 'image_id': 'IMG_20190711_093723.jpg'},
 {'category': 15, 'image_id': 'IMG_20190711_092714.jpg'},
 {'category': 21, 'image_id': 'IMG_20190711_094144.jpg'},
 {'category': 43, 'image_id': 'IMG_20190711_093514.jpg'},
 {'category': 21, 'image_id': 'IMG_20190711_093707.jpg'},
 {'category': 15, 'image_id': 'IMG_20190711_092818.jpg'},
 {'category': 21, 'image_id': 'IMG_20190711_094035.jpg'},
 {'category': 15, 'image_id': 'IMG_20190711_093227.jpg'},
 {'category': 21, 'image_id': 'IMG_20190711_094154.jpg'},
 {'category': 21, 'image_id': 'IMG_20190711_094224.jpg'},
 {'category': 21, 'image_id': 'IMG_20190711_093314.jpg'},
 {'category': 